# 🧱 11 — DIM_IES (Gold)

Este notebook:
- Lê `silver/2018_anonimizado.xlsx` e `silver/2019_anonimizado.xlsx`
- Consolida as bases (2018+2019)
- Constrói a dimensão **DIM_IES**
  - Preferindo `IES_ID_FAKE` / `IES_NOME_FAKE` quando existirem
  - Criando atributos derivados: **PÚBLICA/PRIVADA** e **ÂMBITO** (FEDERAL/ESTADUAL/MUNICIPAL)
- Exporta em `gold/output/dim_ies.csv`


## 0) Imports

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## 1) Paths robustos

In [ ]:
BASE_DIR = Path().resolve()   # pasta atual (gold/)
OUT_DIR = BASE_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_FILES = [
    BASE_DIR / "2018_anonimizado.xlsx",
    BASE_DIR / "2019_anonimizado.xlsx",
]

print("📁 BASE_DIR:", BASE_DIR)
print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f, "| existe?", f.exists())

print("📤 OUT_DIR:", OUT_DIR)


📁 BASE_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold
📥 INPUT_FILES:
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2018_anonimizado.xlsx | existe? True
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2019_anonimizado.xlsx | existe? True
📤 OUT_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output


## 2) Ler Silver (2018 + 2019)

In [3]:
dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)

print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


✅ Linhas/Colunas consolidadas: (732261, 87)


,ULTIMO_PROCESSO,SITUACAO_DO_PROCESSO,IS_SEDE_EAD,NO_DO_PROCESSO,MODALIDADE,ANO_DO_PROTOCOLO,DATA,ORGAO,ATO,CATEGORIA_ATO,...,CINE_AREA_ESPECIFICA,CODIGO_AREA_GERAL_CINE,AREA_GERAL_CINE,CODIGO_AREA_DETALHADA_CINE,AREA_DETALHADA_CINE,CODIGO_AREA_ESPECIFICA_CINE,AREA_ESPECIFICA_CINE,ROTULO_CINE,AVALIACAO_OFICIAL,fonte_arquivo
0,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
1,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
2,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
3,NÃO,Aguardando Pagamento,S,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
4,NÃO,Arquivado,N,20070028,PRESENCIAL,2008,2008-09-26 00:00:00,SERES/DIREG/CGRERCES,Reconhecimento de Curso,Curso,...,Humanidades (exceto línguas),2,Artes e humanidades,223,Filosofia e ética,22,Humanidades (exceto línguas),Filosofia,Regulação,2018_anonimizado.xlsx


## 3) Funções auxiliares

In [4]:
import pandas as pd
import numpy as np

def norm_missing(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip()
    return x.replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})

def dedup_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    score = df_in.notna().sum(axis=1)
    return (df_in.assign(_score=score)
              .sort_values([key, "_score"], ascending=[True, False])
              .drop_duplicates(subset=[key], keep="first")
              .drop(columns=["_score"]))

def pick_first_existing(candidates, df_):
    return next((c for c in candidates if c in df_.columns), None)


## 4) Construir DIM_IES

In [5]:
ies_id_col = pick_first_existing(["IES_ID_FAKE", "CODIGO_DA_IES"], df)
ies_name_col = pick_first_existing(["IES_NOME_FAKE", "NOME_DA_IES"], df)

if ies_id_col is None:
    raise KeyError("Não encontrei coluna de id de IES (IES_ID_FAKE ou CODIGO_DA_IES).")

cols = [ies_id_col]
if ies_name_col is not None:
    cols.append(ies_name_col)

for c in [
    "ORGANIZACAO_ACADEMICA",
    "SISTEMA_DE_ENSINO",
    "CATEGORIA_ADMINISTRATIVA",
    "SITUACAO_DA_IES",
    "UF_PROCESSO",
    "UF_CADASTRO",
    "MUNICIPIO_PROCESSO",
    "MUNICIPIO_CADASTRO",
]:
    if c in df.columns and c not in cols:
        cols.append(c)

dim_ies = df[cols].copy()

dim_ies[ies_id_col] = norm_missing(dim_ies[ies_id_col])
dim_ies = dim_ies.dropna(subset=[ies_id_col])

if ies_name_col is not None:
    dim_ies[ies_name_col] = norm_missing(dim_ies[ies_name_col])

for c in ["ORGANIZACAO_ACADEMICA","SISTEMA_DE_ENSINO","CATEGORIA_ADMINISTRATIVA","SITUACAO_DA_IES"]:
    if c in dim_ies.columns:
        dim_ies[c] = norm_missing(dim_ies[c]).str.upper()

uf_proc = norm_missing(dim_ies["UF_PROCESSO"]) if "UF_PROCESSO" in dim_ies.columns else pd.Series([pd.NA]*len(dim_ies))
uf_cad  = norm_missing(dim_ies["UF_CADASTRO"]) if "UF_CADASTRO" in dim_ies.columns else pd.Series([pd.NA]*len(dim_ies))
dim_ies["UF"] = uf_proc.fillna(uf_cad)

if "CATEGORIA_ADMINISTRATIVA" in dim_ies.columns:
    cat = dim_ies["CATEGORIA_ADMINISTRATIVA"].astype(str).str.upper()
    dim_ies["PUBLICA_PRIVADA"] = np.where(cat.str.contains("PÚBLIC|PUBLIC", na=False), "PÚBLICA", "PRIVADA")
else:
    dim_ies["PUBLICA_PRIVADA"] = "DESCONHECIDO"

if "SISTEMA_DE_ENSINO" in dim_ies.columns:
    sist = dim_ies["SISTEMA_DE_ENSINO"].astype(str).str.upper()
    dim_ies["AMBITO_ADMINISTRATIVO"] = np.select(
        [sist.str.contains("FEDERAL", na=False),
         sist.str.contains("ESTADUAL", na=False),
         sist.str.contains("MUNICIPAL", na=False)],
        ["FEDERAL","ESTADUAL","MUNICIPAL"],
        default="OUTROS"
    )
else:
    dim_ies["AMBITO_ADMINISTRATIVO"] = "DESCONHECIDO"

rename_map = {
    ies_id_col: "id_ies",
    ies_name_col: "nome_ies" if ies_name_col else None,
    "ORGANIZACAO_ACADEMICA": "organizacao_academica",
    "SISTEMA_DE_ENSINO": "sistema_de_ensino",
    "CATEGORIA_ADMINISTRATIVA": "categoria_administrativa",
    "SITUACAO_DA_IES": "situacao_da_ies",
    "UF": "uf",
    "MUNICIPIO_PROCESSO": "municipio_processo",
    "MUNICIPIO_CADASTRO": "municipio_cadastro",
}
rename_map = {k:v for k,v in rename_map.items() if k is not None and v is not None and k in dim_ies.columns}
dim_ies = dim_ies.rename(columns=rename_map)

dim_ies = dedup_most_complete(dim_ies, "id_ies").reset_index(drop=True)

print("✅ DIM_IES pronta:", dim_ies.shape)
dim_ies.head(10)


✅ DIM_IES pronta: (7922, 13)


,id_ies,nome_ies,organizacao_academica,sistema_de_ensino,categoria_administrativa,situacao_da_ies,UF_PROCESSO,UF_CADASTRO,municipio_processo,municipio_cadastro,uf,PUBLICA_PRIVADA,AMBITO_ADMINISTRATIVO
0,1,UNIVERSIDADE FEDERAL DE MATO GROSSO - UFMT,UNIVERSIDADE,FEDERAL,PÚBLICA FEDERAL,ATIVA,MT,MT,Pontal do Araguaia,Pontal do Araguaia,MT,PÚBLICA,FEDERAL
1,10,PONTIFICIA UNIVERSIDADE CATOLICA DO PARANA - P...,UNIVERSIDADE,FEDERAL,PRIVADA SEM FINS LUCRATIVOS,ATIVA,PR,PR,Curitiba,Curitiba,PR,PRIVADA,FEDERAL
2,10001,FACULDADE XYZ - FACXYZ,FACULDADE,FEDERAL,PRIVADA COM FINS LUCRATIVOS,NaN,DF,NaN,Brasília,NaN,DF,PRIVADA,FEDERAL
3,10009,FAMAC - FACULDADE DE MACHADINHO DO OESTE - FAMAC,FACULDADE,FEDERAL,PRIVADA COM FINS LUCRATIVOS,EXTINTA,RO,RO,Machadinho D´Oeste,Machadinho D´Oeste,RO,PRIVADA,FEDERAL
4,10010,FACULDADE SENAC CONCORDIA - FACSENAC,FACULDADE,FEDERAL,PRIVADA SEM FINS LUCRATIVOS,NaN,SC,NaN,Concórdia,NaN,SC,PRIVADA,FEDERAL
5,10011,FACULDADE SENAC CRICIUMA - FACSENAC,FACULDADE,FEDERAL,PRIVADA SEM FINS LUCRATIVOS,NaN,SC,NaN,Criciúma,NaN,SC,PRIVADA,FEDERAL
6,10012,FACULDADE SENAC SAO MIGUEL DO OESTE - FACSENAC,FACULDADE,FEDERAL,PRIVADA SEM FINS LUCRATIVOS,NaN,SC,NaN,São Miguel do Oeste,NaN,SC,PRIVADA,FEDERAL
7,10013,NaN,FACULDADE,FEDERAL,PRIVADA SEM FINS LUCRATIVOS,NaN,SC,NaN,Caçador,NaN,SC,PRIVADA,FEDERAL
8,10016,FACULDADE OBOE - FACO - FACO,FACULDADE,FEDERAL,PRIVADA COM FINS LUCRATIVOS,EXTINTA,CE,CE,Fortaleza,Fortaleza,CE,PRIVADA,FEDERAL
9,10020,INSTITUTO LOGOS DE EDUCACAO SUPERIOR - ILES,FACULDADE,FEDERAL,PRIVADA COM FINS LUCRATIVOS,EXTINTA,AL,AL,São José da Tapera,São José da Tapera,AL,PRIVADA,FEDERAL


## 5) Exportar

In [6]:
out_file = OUT_DIR / "dim_ies.csv"
dim_ies.to_csv(out_file, index=False, encoding="utf-8")
print("✅ Salvo em:", out_file)


✅ Salvo em: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output\dim_ies.csv
